In [1]:
import os
from dotenv import load_dotenv

dotenv_path = '.env'
load_dotenv(dotenv_path)

# Print out the environment variables loaded
for key, value in os.environ.items():
    if 'OPENAI' in key:
        print("OpenAI Key founded")
        #print(f"{key}: {value}")

openai_api_key = os.getenv('OPENAI_API_KEY')

#print(f"OpenAI API Key: {openai_api_key}")

OpenAI Key founded


# Describe the functions

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.openai_functions import create_openai_fn_chain
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [4]:
llm = ChatOpenAI()

In [5]:
prompt = ChatPromptTemplate.from_template(
    "Tell me the name of the captain of the team who won the soccer world cup of {year}?"
)

In [ ]:
output_parser = JsonOutputFunctionsParser()

In [7]:
functions_soccer = [
    {
      "name": "player_full_name",
      "description": "Name and lastname of the player",
      "parameters": {
        "type": "object",
        "properties": {
          "name": {
            "type": "string",
            "description": "The first name of the player"
          },
          "lastname": {
            "type": "string",
            "description": "The lastname of the player"
          }
        },
        "required": ["name", "lastname"]
      }
    }
  ]

In [10]:
new_chain = (
    prompt 
    | llm.bind(function_call={"name": "player_full_name"}, functions=functions_soccer)
    | JsonOutputFunctionsParser()
)
new_chain.invoke({"year": "2010"})

{'name': 'Iker', 'lastname': 'Casillas'}

## Other Example 

In [41]:
#!pip uninstall -y openai 
#!pip install openai==1.0.0

In [15]:
#import openai
#print(openai.__version__)


In [2]:
from langchain.prompts import ChatPromptTemplate

prompt_salon = ChatPromptTemplate.from_template(
    "Eres el AI chatbot del salon teniendo una conversacion con el humano.\n\n"
    "Humano: {human_input}\n"
    "AI:"
)

In [3]:
functions_salon = [
    {
        "name": "get_item_info",
        "description": "Obtener el nombre y precio de los productos del catalogo del salon de belleza X",
        "parameters": {
            "type": "object",
            "properties": {
                "nombre_producto": {
                    "type": "string",
                    "description": "El nombre del producto del catalogo, por ejemplo, Shampoo anticaspa",
                },
            },
            "required": ["nombre_producto"],
        },
    },
    {
        "name": "place_order",
        "description": "Realizar un pedido de un producto del catalogo del salon de belleza X",
        "parameters": {
            "type": "object",
            "properties": {
                "nombre_producto": {
                    "type": "string",
                    "description": "El nombre del producto que desea pedir, por ejemplo, Shampoo anticaspa",
                },
                "cantidad": {
                    "type": "integer",
                    "description": "La cantidad de productos que desea pedir",
                    "minimum": 1
                },
                "direccion": {
                    "type": "string",
                    "description": "La direccion donde se debe entregar la comida",
                },
            },
            "required": ["nombre_producto", "cantidad", "direccion"],
        },
    }
]

In [4]:
fake_db = {
    "products": {
        "Shampoo anticaspa": {"price": 15.00, "ingredients": ["shampoo", "aloe", "mental"]},
        "Tinte rojo": {"price": 18.00, "ingredients": ["colorizante", "color amarillo", "agua"]},
        "Peluca": {"price": 12.50, "ingredients": ["cabello", "pegamento", "fibra plastica"]}
    },
    "orders": []
}

In [5]:
def get_item_info(nombre_producto):
    item = fake_db["products"].get(nombre_producto)
    if not item:
        return f"No tenemos informacion de ese producto ({nombre_producto})."
    return f"Nombre: {nombre_producto}, Precio: {item['price']}, Ingredientes: {item['ingredients']}"

def place_order(nombre_producto, cantidad, direccion):
    item = fake_db["products"].get(nombre_producto)
    if not item:
        return f"No tenemos ese producto ({nombre_producto})."
    
    order_id = len(fake_db["orders"]) + 1
    order = {
        "id": order_id,
        "product": nombre_producto,
        "quantity": cantidad,
        "address": direccion,
        "total_price": item["price"] * cantidad
    }
    fake_db["orders"].append(order)
    return f"Hemos puesto tu orden! Tu numero de orden es {order_id}. El precio total es {order['total_price']}."

In [8]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [9]:
from langchain_core.runnables import RunnableSequence
#from langchain_community.chat_models import OpenAIChat
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [9]:
# Define prompt_salon to accept and process human_input
def prompt_salon(human_input):
    return f"Prompt: {human_input}"

chat = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=openai_api_key)
output_parser_salon = JsonOutputFunctionsParser()

# Assuming `functions_salon` is properly defined elsewhere in your code
# If it isn't, you'll need to define it accordingly.

new_chain = (
    prompt_salon
    | chat.bind(function_call={"name": "place_order"}, functions=functions_salon)
    | output_parser_salon
)
# Invoke the chain with the correct variable name
response = new_chain.invoke({"human_input": "Quiero ordenar tres Tinte rojo para llevar a Plaza Naco, cual es precio?"})
print(response)

{'nombre_producto': 'Tinte rojo', 'cantidad': 3, 'direccion': 'Plaza Naco'}
